# Projeto 1 - Ciência dos Dados

Nome: Gustavo Katz

Nome: Rafael Oncins

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [15]:
import os

filename = 'Fall guys.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Fall guys.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [16]:
train = pd.read_excel(filename)
train = train.loc[:,['Treinamento', 'Relevância']]
train.head(5)

,Treinamento,Relevância
0,"atualizaram o fall guys e ficou uma merda, par...",1
1,"hj tem live, so q cai ser uma curta. @mrsimpso...",0
2,@jaoreg racho com os vídeos dele de fall guys,1
3,"eu jogando fall guys é mt sarro, eu dou uns gr...",0
4,adicionaram easy anti cheat no fall guys vitór...,1


In [17]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,confira minha transmissão do meu playstation 4...,1
1,fall guys ta todo lagado so queria jogar,1
2,o jeito q eu sou péssima no fall guys é difere...,0
3,@marvinsantosl1 enquanto fall guys não faz cro...,0
4,rt @2albafica1241: vou fazer a live de fall gu...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é o jogo de videogame Fall Guys. O nosso critério para tweets relevantes corresponde ao tipo de conteúdo que eles tinham, na qual a gente focou em comentários críticos sobre o jogo (com relação a atualizações e bugs), lives e streamings (que geralmente atraem pessoas que se interessam e geram críticas importantes), e qualquer tipo de comentário com relação ao servidor e conteúdos do sistema.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [18]:
import re 

#função de limpeza removendo as pontuações
def cleanup(text):
    punctuation = '[!-.:,:?;/@")(|,[\n]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [19]:
#criando uma lista de listas em que as sublistas são os tweets e os itens são as palavras
LISTAO = []

for i in train.Treinamento: 
    tweet_geral = cleanup(i)
    palavras_geral = tweet_geral.split()
    LISTAO.append(palavras_geral)

In [20]:
#criando uma lista com todas as palavras de todos os tweets
lista_tweets_geral = []

for i in train.Treinamento: 
    tweet_geral = cleanup(i)
    palavras_geral = tweet_geral.split()
    lista_tweets_geral.extend(palavras_geral)

In [21]:
#criando uma lista com todas as palavras de todos tweets sem repetição
lista_tweets_SR = []
for palavra in lista_tweets_geral:
    if palavra not in lista_tweets_SR:
        lista_tweets_SR.append(palavra)
        
#número de palavras em todos os tweets
quant_total = len(lista_tweets_SR)
print(f'Quantidade total de palavras nos tweets de treinamento sem repetição: {quant_total}')

Quantidade total de palavras nos tweets de treinamento sem repetição: 1714


### TWEETS RELEVANTES

In [22]:
#criando o pd.Series de tweets relevantes
relevante = train.Relevância == 1
tweets_relevantes = train.loc[relevante, 'Treinamento']


#aplicando a função de limpeza e criando uma lista com as palavras dos tweets relevantes
lista_relev = []

for i in tweets_relevantes:
    tweet_relev = cleanup(i)
    palavras_relev = tweet_relev.split()
    lista_relev.extend(palavras_relev) #lista de palavras nos tweets relevantes
    

In [23]:
#criando uma lista com todas as palavras dos tweets relevantes sem repetição
lista_relevSR = []
for palavra in lista_relev:
    if palavra not in lista_relevSR:
        lista_relevSR.append(palavra)
        

#número de palavras nos tweets relevantes
quant_R = len(lista_relevSR)
print(f'Quantidade total de palavras nos tweets relevantes de treinamento sem repetição: {quant_R}')

Quantidade total de palavras nos tweets relevantes de treinamento sem repetição: 911


### TWEETS NÃO RELEVANTES

In [24]:
#criando o pd.Series de tweets não relevantes
naorelevante = train.Relevância == 0
tweets_naorelevantes = train.loc[naorelevante, 'Treinamento']

#aplicando a função de limpeza e criando uma lista com as palavras dos tweets não relevantes
lista_naorelev = []

for i in tweets_naorelevantes:
    tweet_naorelev = cleanup(i)
    palavras_naorelev = tweet_naorelev.split()
    lista_naorelev.extend(palavras_naorelev) #lista de palavras nos tweets não relevantes
    

In [25]:
#criando uma lista com todas as palavras dos tweets relevantes sem repetição
lista_naorelevSR = []
for palavra in lista_naorelev:
    if palavra not in lista_naorelevSR:
        lista_naorelevSR.append(palavra)
        

#número de palavras nos tweets não relevantes
quant_NR = len(lista_naorelevSR)
print(f'Quantidade total de palavras nos tweets não relevantes de treinamento sem repetição: {quant_NR}')

Quantidade total de palavras nos tweets não relevantes de treinamento sem repetição: 1096


### CLASSIFICADOR

In [26]:
#Essa célula é um teste do nosso classificador, de modo que foi utilizada a própria planilha de treinamento
#com o objetivo de verificar se os resultados seriam razoáveis

#probabilidades
prob_r = 0.41111         # P(relevante)
prob_nr = 1 - 0.41111    # P(não relevante)

TR = []                  # lista com os tweets classificados como relevantes
TNR = []                 # lista com os tweets classificados como não relevantes
classificacao = []       # lista de classificação como R (relevante) ou NR (não relevante)


# classificador
for tweet in LISTAO: 
    x=1
    y=1
    
    for palavra in tweet:
        a = lista_relev.count(palavra)
        prob_a_dado_r =  (a + 1)/(quant_R + quant_total)
        x = x*prob_a_dado_r
       
        b = lista_naorelev.count(palavra)
        prob_b_dado_nr = (b + 1)/(quant_NR + quant_total)
        y = y*prob_b_dado_nr
    
    prob_r_dado_tweet = prob_r*x
    prob_nr_dado_tweet = prob_nr*y

    if prob_r_dado_tweet > prob_nr_dado_tweet:
        TR.append(tweet)
        classificacao.append('R')
    else:
        TNR.append(tweet)
        classificacao.append('NR')
         

In [27]:
print(f'Número de tweets classificados como relevantes: {len(TR)}')
print(f'Número de tweets classificados como não relevantes: {len(TNR)}')

Número de tweets classificados como relevantes: 138
Número de tweets classificados como não relevantes: 222


In [28]:
#tabela de classificação do nosso modelo de validação do classificador
pd.crosstab(train.Relevância, pd.Series(classificacao))

col_0,NR,R
Relevância,,
0,211,1
1,11,137


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [29]:
#aplicando a função de limpeza nos tweets da parte teste

lista_teste = []      #lista de todos os tweets do parte teste
LISTAO_TESTE = []     #lista de listas em que as sublistas são os tweets e os itens são as palavras da parte teste

for i in test.Teste: 
    tweet_teste = cleanup(i)
    lista_teste.append(tweet_teste)
    palavras_teste = tweet_teste.split()
    LISTAO_TESTE.append(palavras_teste)
    
print(f'Número de tweets utilizados no teste: {len(lista_teste)}')

Número de tweets utilizados no teste: 171


In [30]:
#classificador
c_teste = []     # lista de classificação como R (relevante) ou NR (não relevante)

for tweet in LISTAO_TESTE: 
    x=1
    y=1
    
    for palavra in tweet:
        a = lista_relev.count(palavra)
        prob_a_dado_r =  (a + 1)/(quant_R + quant_total)
        x = x*prob_a_dado_r
       
        b = lista_naorelev.count(palavra)
        prob_b_dado_nr = (b + 1)/(quant_NR + quant_total)
        y = y*prob_b_dado_nr
    
    prob_r_dado_tweet = prob_r*x
    prob_nr_dado_tweet = prob_nr*y

    if prob_r_dado_tweet > prob_nr_dado_tweet:
        c_teste.append('R')
    else:
        c_teste.append('NR')

In [31]:
#tabela de classificação final da planilha de teste
pd.crosstab(test.Relevância, pd.Series(c_teste), normalize = True)

col_0,NR,R
Relevância,,
0,0.578947,0.040936
1,0.128655,0.251462


In [33]:
pd.crosstab(test.Relevância, pd.Series(c_teste), normalize = 'index')

col_0,NR,R
Relevância,,
0,0.933962,0.066038
1,0.338462,0.661538


In [36]:
print ('Para avaliar a eficiência do nosso classificador, montamos a tabela cruzada acima.')
print ('Com ela, foi possível encontrar as porcentagens de acerto e de erro do classificador.')
print('-'*100)
print('- verdadeiros positivos: 25,1%    (mensagens relevantes classificadas como relevantes)')
print('- falsos positivos: 4,1%          (mensagens irrelevantes que são classificadas como relevantes)')
print('- verdadeiros negativos: 57,8%    (mensagens irrelevantes que são classificadas como irrelevantes)')
print('- falsos negativos: 12,9%         (mensagens relevantes que são cassificadas como irrelevantes)')
print('-'*100)
print('- Porcentagem de acerto dos tweets relevantes: 66,2%')
print('- Porcentagem de acerto dos tweets irrelevantes: 93,4%')

Para avaliar a eficiência do nosso classificador, montamos a tabela cruzada acima.
Com ela, foi possível encontrar as porcentagens de acerto e de erro do classificador.
----------------------------------------------------------------------------------------------------
- verdadeiros positivos: 25,1%    (mensagens relevantes classificadas como relevantes)
- falsos positivos: 4,1%          (mensagens irrelevantes que são classificadas como relevantes)
- verdadeiros negativos: 57,8%    (mensagens irrelevantes que são classificadas como irrelevantes)
- falsos negativos: 12,9%         (mensagens relevantes que são cassificadas como irrelevantes)
----------------------------------------------------------------------------------------------------
- Porcentagem de acerto dos tweets relevantes: 66,2%
- Porcentagem de acerto dos tweets irrelevantes: 93,4%


___
### Concluindo

- PERFORMANCE:
O nosso classificador teve uma boa performance, mostrando uma alta porcentagem de acerto nos tweets irrelevantes , e uma porcentagem razoável de acerto nos tweets relevantes da planilha de teste. Na classificação dos tweets irrelevantes, houve uma porcentagem de cerca de 90% de acerto, já na classificação dos tweets relevantes, cerca de 2/3 dos tweets foram classificados corretamente. 

- SARCASMO E DUPLA NEGAÇÃO:
Como o nosso classificador se baseia no número de vezes que uma palavra aparece em determinado tweet, os comentário que envolvem sarcasmo e dupla negação não são possíveis de serem classificados utilizando esse método. Isso acontece, pois o classificador não consegue identificar o sentido e o significado da frase, apenas as palavras que estão presentes no comentário.

- PLANO DE EXPANSÃO E OUTRAS OPÇÕES DE UTILIZAÇÃO DO CLASSIFICADOR:
Apesar do classificador ter obtido uma boa porcentagem de acerto, ainda há maneiras de tornar o código mais preciso e trazer resultados melhores. Esse método poderia ser aplicado em outras redes sociais, com o mesmo objetivo, a identificação de posts relevantes. E também em diferentes cenários, por exemplo: diagnósticos médicos, identificação de e-mails como spam, classificação de artigos em diferentes temas, software de reconhecimento facial, entre outros.

- POSSÍVEIS MELHORIAS:
Uma maneira de melhorar a performance do nosso classificador, seria utilizar como base para classificação os radicais das palavras, o que evitaria a diferenciação de verbos conjugados de diferentes maneiras, ou palavras com afixos diferentes. Para isso, seria possível utilizar uma função que identifica os radicais. Poderia utilizar a técnica do stemming, utlizando a biblioteca nltk.
        
Esses dados foram obtidos no site: https://minerandodados.com.br/analise-de-sentimentos-utilizando-dados-do-twitter/

- Por que não utilizar o próprio classificador para gerar mais amostras de treinamento?

Não é possível utilizar o próprio classificador para gerar mais amostras de treinamento, pois poderia entrar em um tipo de looping, uma vez que essas novas amostras entrariam no classificador podendo levar a um escalonamento de erros, de modo que a qualidade do classificador e a sua validade ficariam comprometidas.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**